# Apresentação

Notebook destinado ao estudo da implementação de tarefas automatizadas com CrewAI, um framework relacionado à criação de Agents. O principal foco desse notebook referente aos demais no repositório é a formatação da saída e seu envio, bem como na criação de tools customizadas, que podem atender aos interesses do desenvolvedor e ao contexto de aplicação.

## Biblioteca


In [5]:
!pip install crewai langchain_google_genai -q

!pip install google-generativeai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 39.1 MB/s eta 0:00:00


In [6]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 -q

In [7]:
import os
import crewai
import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI

from crewai import Agent, Task, Crew

from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool

from crewai_tools import BaseTool
from pydantic import BaseModel

In [8]:
os.environ["SERPER_API_KEY"] = ""

os.environ["GOOGLE_API_KEY"] = ""

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [9]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.5
)

In [10]:
# Testando o modelo.

llm.predict("hi.")

'Hi there! What can I do for you today? \n'

In [11]:
# Inicializando as ferramentas a serem usadas pelos Agents

serper_tool = SerperDevTool()
scrape_tool_local = ScrapeWebsiteTool()

In [12]:
# Exemplo de uma tool.

class SentimentAnalysisTool(BaseTool):

  name : str = "Sentiment Analysis Tool"
  description: str= ("""Utilize da ferramenta de busca de conteúdo na
                        internet para procurar sobre acertos e defeitos
                        de histórico de eventos no impacto do sentimento do público.""")

  def _run(self, text : str) -> str:

    return "positive"

In [13]:
sentiment_analyses_tool = SentimentAnalysisTool()

In [14]:
# Agente - 1 : coordenador de local.

venue_coordinator = Agent(
    role = "Venue Coordinator",
    goal = ("""Identify and book an appropriate venue
            based on event requirements"""),
    tools = [serper_tool, scrape_tool_local],
    backstory = (
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    allow_delegation = False,
    verbose = True,
    llm = llm,
)

In [15]:
# Agent - 2 : Gerente de logística

logistics_manager = Agent(
    role = "Logistics Manager",
    goal = ("Manage all logistics for the event "
           "including catering and equipment"),
    tools = [serper_tool, scrape_tool_local],
    backstory = (
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    allow_delegation = False,
    verbose = True,
    llm = llm

)

In [16]:
# Agent - 3 : Comunicação e Marketing

marketing_communication_agent = Agent(
    role = "Marketing and Communications Agent",
    goal = (
        "Effectively market the event and "
        "communicate with participants"
    ),
    tools = [serper_tool, scrape_tool_local],
    backstory = (
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    allow_delegation = False,
    verbose = True,
    llm = llm
)

In [17]:
# Criando uma saída customizada. Em alguns cenários pode ser útil que
# seja utilizada uma saída estruturada e sintética. Assim, pode-se criar
# uma determinada especificação, de modo a informar a LLM para que gere
# a informação no formato especificado.

class VenueDetails(BaseModel):

  name : str
  address : str
  capacity : int
  booking_status : str


## Tarefas

In [18]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [19]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [20]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communication_agent
)

## Criando a Crew

In [21]:
# Definindo a Crew com os Agents.

event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communication_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True,

    memory = True,

    embedder={
        "provider": "google",
            "config": {
                "model": "models/embedding-001",
                "task_type": "retrieval_document",
                "title": "Embeddings for Embedchain"
            }
    }

)

In [22]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "São Paulo",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [23]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in São Paulo that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to start researching venues online.
Action: Search the internet
Action Input: {"search_query": "tech venues in São Paulo"} 


Search results: Title: IT & Technology Events in São Paulo - Brazil - 10Times
Link: https://10times.com/saopaulo-br/technology
Snippet: IT & Technology Events in São Paulo · Eletrolar Show · NetCom · User Conference Brasil · Brazil Promotion · IDUG Latin America Db2 Tech Conference · Digitalks Expo.
---
Title: Science & Tech: High Tech Events & Tickets in São Paulo, Brazil
Link: https://www.eventbrite.ie/b/brazil--s%C3%A3o-paulo/science-and-tech/high-tech/
Snippet: High Tech events. in São Paulo, Brazil ; Mind The Sec 2024. Tue, 17 Sept, 08:00. Transamerica Expo Center. From R$199.00 ; Cyber Security Summit 2024. Mon, 28 Oct ...
---
Title: São Paulo Expo 

Thought: I need to find a venue that specifically mentions tech or innovation and can accommodate a conference.
Action: Read website content
Action Input: {"website_url": "https://www.qmctelecom.com/indoor-solutions/case-sao-paulo-expo/"} 

São Paulo Expo - QMC Telecom Skip to content
 About UsInvestorsCareersMedia
 MenuAbout UsInvestorsCareersMedia
flag-usa WHAT WE DOINDOOR SOLUTIONSOUTDOOR SOLUTIONSSLS – STREET LEVEL SOLUTIONSSUCCESS CASES
 MenuWHAT WE DOINDOOR SOLUTIONSOUTDOOR SOLUTIONSSLS – STREET LEVEL SOLUTIONSSUCCESS CASES
 WHAT WE DOINDOOR SOLUTIONSOUTDOOR SOLUTIONSSLS – STREET LEVEL SOLUTIONSSUCCESS CASES
 MenuWHAT WE DOINDOOR SOLUTIONSOUTDOOR SOLUTIONSSLS – STREET LEVEL SOLUTIONSSUCCESS CASES
Contact Us
São Paulo ExpoDETAILS DRIVE CONNECTIVITY EXCELLENCE
Expanded and renamed Sao Paulo Expo, the former lmigrantes Exhibition & Convention Center has become one of the most sought-after venues for events in Brazil and one of the largest and best­ structured exhibition centers in L

Thought: Inovabra seems like a promising venue for a Tech Innovation Conference. I should check their website for more details.
Action: Read website content
Action Input: {"website_url": "https://inovabra.com.br/"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





Thought: I need to find a way to get more information about Inovabra's event space.
Action: Search the internet
Action Input: {"search_query": "Inovabra Sao Paulo event space"} 


Search results: Title: 25 Best Venues for Business Events in São Paulo
Link: https://venueful.com/en/brazil/best-venues-for-business-events-in-sao-paulo
Snippet: Hold your event in one of the city's top postcards. Located on Paulista Avenue, MASP is the perfect venue for concerts, shows and corporate ...
---
Title: Inovabra in São Paulo - Venueful
Link: https://venueful.com/en/brazil/sao-paulo/inovabra
Snippet: A co-innovation space of 22000 m² which is the perfect translation of the inovabra ecosystem.
---
Title: São Paulo Event Space | Renaissance São Paulo Hotel - Marriott
Link: https://www.marriott.com/en-us/hotels/saobr-renaissance-sao-paulo-hotel/events/
Snippet: Host your meeting or gathering at Renaissance São Paulo Hotel for impeccable services, elegant wedding venues, and event spaces in São Paulo, 

This is the agent final answer: Inovabra in São Paulo
Select language
Português
English
Brazil
São Paulo
Inovabra
Inovabra
										A co-innovation space of 22,000 m² which is the perfect translation of the inovabra ecosystem.
										Here, companies, startups, investors, mentors and entrepreneurs generate new business and seek innovative solutions based on networking and collaboration.
										Venue Overview
												Venue Type:
Auditorium
Public Venue
												Capacity:
150 people total
150 people in the same space
												Price Level:
$ $ $
										Facilities
Furnitures
Audio / Video
Internet
Air Conditioning
Open Air Space
Parking
Kitchen
Expo Area
Accommodation
Stadium Seating
Rooftop
Wheelchair Accessibility
										Location
												Address:
													Av. Angélica, nº 2529
													Bela Vista
													01227-200 - São Paulo -
												Neighbourhood:
													Consolação
								x
										Find a venue in São Paulo
										We provide FREE 

Thought: The São Paulo Expo venue mentioned in the search results seems like it could accommodate a large tech conference. 
Action: Read website content
Action Input: {"website_url": "https://www.saopauloexpo.com.br/en"} 

São Paulo Expo | Site
EVENT PLANNER
EXHIBITOR
VISITOR
ESG/CSR
EN
PT
EN
PT
VISITOR
EXHIBITOR
EVENT PLANNER
ESG/CSR
//  About São Paulo Expo
// Events Schedule
// Photos
//  Location
// Press
// Contact
// Parking Lot
// Facilities
// Food & Beverages
// Modular Rooms
// Merchandising
// Plan your Visit
// Destination Santos
// Versatile Spaces
// Facilities
// Food & Beverages
// Why São Paulo
// Merchandising
// Parking Lot
// Plan your Visit
// Parking Lot
// Gastronomy
// Events Schedule
// Explore São Paulo
//  About São Paulo Expo
// Events Schedule
// Photos
//  Location
// Press
// Contact
// Parking Lot
// Facilities
// Food & Beverages
// Modular Rooms
// Merchandising
// Plan your Visit
// Destination Santos
// Versatile Spaces
// Facilities
// Food & Bevera

Thought: The search results show Sao Paulo Expo has a large capacity and is suitable for large events. I will use the 10times link to get more details.
Action: Read website content
Action Input: {"website_url": "https://10times.com/venues/sao-paulo-expo-176655"} 

Attention Required! | Cloudflare
Please enable cookies.
Sorry, you have been blocked
You are unable to access 10times.com
Why have I been blocked?
This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.
What can I do to resolve this?
You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.
Cloudflare Ray ID: 8a2301d8e9e28476
•
 Your IP:
 Click to reveal
34.80.182.26
•
Performance & se

Thought: I am unable to access the content on 10times.com due to security measures. I will try another link from the search results to get more details about São Paulo Expo.
Action: Read website content
Action Input: {"website_url": "https://www.gl-events.com/en/sao-paulo-expo"} 

Sao Paulo Expo | GL events Venues
 Skip to main content
Secondary navigation
News
Investors Relation
Careers
Pressroom
ESG/CSR
Navigation header
Group
References
Live
Venues
Exhibitions
Contact
EnglishFrenchChinesePortugueseSpanish
Navigation header
Group
References
Live
Venues
Exhibitions
Contact
Secondary navigation
News
Investors Relation
Careers
Pressroom
ESG/CSR
EnglishFrenchChinesePortugueseSpanish
São Paulo Expo
São Paulo, Brazil
Back
Inaugurated in April 2016, São Paulo Expo is one of the most modern and efficient exhibition and convention centres in Latin America. Following major expansion and renovation work, it now boasts 90,000 sqm of indoor surface area, divided into 8 modular pavilions, and a 10

Thought: I now have enough information about São Paulo Expo to present it as a venue option.
Final Answer: Sao Paulo Expo

Inaugurated in April 2016, São Paulo Expo is one of the most modern and efficient exhibition and convention centres in Latin America. Following major expansion and renovation work, it now boasts 90,000 sqm of indoor surface area, divided into 8 modular pavilions, and a 10,000 sqm convention centre. 

The venue is located 3 km from the international airport. It is designed to be able to host several events at the same time and offers many facilities and services to organisers and visitors, notably 4,500-space covered car park, which is unique in Brazil.

Capacities
-  Total exhibition space: 100,000 sqm
-  Auditorium: from 70 to 7,810 seats

Types of events hosted
Trade fairs, congresses, conventions, seminars, gala dinners, concerts, sports events, etc.

Event layout
-    Congress : up to 7,810 people
-  Convention : up to 4,000 people
-  Dinner : up to 3,000 peopl

In [24]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)


In [25]:
data

{'name': 'Sao Paulo Expo',
 'address': 'Rodovia dos Imigrantes, km 1,5\nCEP: 04329-900 São Paulo SP, Brazil',
 'capacity': 100000,
 'booking_status': 'Not Available'}

In [26]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md